In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download delayedkarma/impressionist-classifier-data
! unzip impressionist-classifier-data.zip

100% 2.19G/2.19G [01:38<00:00, 25.6MB/s]
100% 2.19G/2.19G [01:38<00:00, 23.9MB/s]
Archive:  impressionist-classifier-data.zip
  inflating: training/training/Cezanne/215457.jpg  
  inflating: training/training/Cezanne/215458.jpg  
  inflating: training/training/Cezanne/215459.jpg  
  inflating: training/training/Cezanne/215460.jpg  
  inflating: training/training/Cezanne/215462.jpg  
  inflating: training/training/Cezanne/215463.jpg  
  inflating: training/training/Cezanne/215466.jpg  
  inflating: training/training/Cezanne/215467.jpg  
  inflating: training/training/Cezanne/215468.jpg  
  inflating: training/training/Cezanne/215469.jpg  
  inflating: training/training/Cezanne/215470.jpg  
  inflating: training/training/Cezanne/215471.jpg  
  inflating: training/training/Cezanne/215473.jpg  
  inflating: training/training/Cezanne/215474.jpg  
  inflating: training/training/Cezanne/215475.jpg  
  inflating: training/training/Cezanne/215476.jpg  
  inflating: training/training/Cezanne/215

In [4]:
import numpy as np
import pandas as pd
import os
import torchvision
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torch import Generator
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import itertools
import time
import copy
from torchvision.models import ResNet50_Weights
from torch.nn import Module
from torchvision import models
import torch.optim as optim
import seaborn as sns
import torchvision
import torch.nn as nn
from torch import manual_seed as torch_manual_seed
from torch.cuda import max_memory_allocated, set_device, manual_seed_all
from torch.backends import cudnn

In [6]:
def setup_seed(seed):
    torch_manual_seed(seed)
    manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    cudnn.deterministic = True

SEED = 6050
setup_seed(SEED)

In [5]:
artists = ['Cezanne', 'Degas', 'Gauguin', 'Hassam', 'Matisse', 'Monet', 'Pissarro', 'Renoir', 'Sargent', 'VanGogh']
artists = os.listdir('training/training')

In [7]:
transformation = Compose([Resize((256,256)), ToTensor()])
transformation_train = Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(20),
    ToTensor()
])

image_datasets = {}
image_datasets['training'] = ImageFolder(f'/content/training/training', transform=transformation_train)
image_datasets['validation'] = ImageFolder(f'/content/validation/validation', transform=transformation)

size_all_validation = len(image_datasets['validation'])
size_test_from_validation = int(size_all_validation * 0.5)
size_validation = size_all_validation - size_test_from_validation

image_datasets['validation'], image_datasets['testing'] = random_split(image_datasets['validation'], [size_validation, size_test_from_validation], 
                                                                       generator=Generator().manual_seed(SEED))

BS = 32
dataloaders = {x: DataLoader(image_datasets[x], batch_size=BS, shuffle=True) for x in ['training', 'validation', 'testing']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['training', 'validation', 'testing']}
print(dataset_sizes)

# set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

{'training': 3988, 'validation': 495, 'testing': 495}
cuda:0


## Model 1: ResNet50

In [26]:
# define the model to have the pre-trained resnet 50 parameters
model1 = torchvision.models.resnet50(weights=ResNet50_Weights.DEFAULT)
# freeze all of the parameters in the model
for param in model1.parameters():
  param.requires_grad = False
# unfreeze the parameters in the last residual block of the architecture
for name, param in model1.named_parameters():
  for i in [4]:
    if name.startswith(f'layer{i}'):
      param.requires_grad = True
# EDIT DROPOUT RATE HERE
DO = 0.0
# construct the fully connected head which will receive the flattened convolutional output
model1.fc = nn.Sequential(
               nn.Linear(2048, 512),
               nn.BatchNorm1d(512),
               nn.ReLU(inplace=True),
               nn.Dropout(DO),
               
               nn.Linear(512, 128),
               nn.BatchNorm1d(128),
               nn.ReLU(inplace=True),
               nn.Dropout(DO),

               nn.Linear(128, len(artists)))

# load the model to device
model1 = model1.to(device)

In [36]:
path = f'/content/drive/My Drive/resnetModel.pt'
model1.load_state_dict(torch.load(path))

<All keys matched successfully>

## Model 2: EfficientNet

In [28]:
SEED = 6050
setup_seed(SEED)
from torchvision.models import EfficientNet_B0_Weights

# Load the pre-trained EfficientNet-B0 model
model2 = torchvision.models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)

# Freeze all layers except the last two blocks
for name, param in model2.named_parameters():
    param.requires_grad = False

for name, param in model2.named_parameters():
  for i in [7,8]:
    if name.startswith(f'features.{i}'):
      param.requires_grad = True

# # Modify the fully connected head
DO = 0.
model2.classifier = nn.Sequential(
    nn.Linear(1280, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(inplace=True),
    nn.Dropout(DO),

    nn.Linear(256, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(inplace=True),
    nn.Dropout(DO),

    nn.Linear(64, len(artists))
)

# Load the model to device
model2 = model2.to(device)

In [29]:
path = f'/content/drive/My Drive/efficientnetModel.pt'
model2.load_state_dict(torch.load(path))

<All keys matched successfully>

## Model 3: GoogLeNet

In [30]:
!pip install googlenet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
# batch_size = 32, learning_rate = 0.0005, weight_decay = 1e-5, Data Augmentation

from googlenet_pytorch import GoogLeNet
model3 = GoogLeNet.from_pretrained("googlenet")

# freeze all of the parameters in the model
for param in model3.parameters():
    param.requires_grad = False
# unfreeze the parameters in the last residual block of the architecture
for name, param in model3.named_parameters():
    for i in [5]:
        if name.startswith(f'inception{5}') or name.startswith('aux'):
            param.requires_grad = True

# EDIT DROPOUT RATE HERE (dropout actually doesn't help here, so let's use L2 regularization instead)
DO = 0.0
# construct the fully connected head which will receive the flattened convolutional output

'''
The final layer of GoogLeNet is a global average pooling layer that reduces the spatial 
dimensions of the feature maps to 1x1 and produces a tensor of size (1, 1024).
''' 
model3.fc = nn.Sequential(
               nn.Linear(1024, 512), # 2048 -> 1024
               nn.BatchNorm1d(512),
               nn.ReLU(inplace=True),
               nn.Dropout(DO),
               
               nn.Linear(512, 128),
               nn.BatchNorm1d(128),
               nn.ReLU(inplace=True),
               nn.Dropout(DO),

               nn.Linear(128, len(artists)))

model3 = model3.to(device)
model3.aux_logits = False ### NEVER REMOVE THIS LINE

Loaded pretrained weights for googlenet


In [32]:
path = f'/content/drive/My Drive/googlenetModel.pt'
model3.load_state_dict(torch.load(path))

<All keys matched successfully>

##**Final Ensemble**

In [37]:
# define the test loop
def ensemble_test_loop(dataloader, model1, model2, model3, loss_fn):
  # calculate the size of the dataset and the number of batches
  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  # initialize the model performance metrics
  test_loss, correct = 0, 0

  # set model to evaluate model
  model1.eval()
  model2.eval()
  model3.eval()

  # iterate over the data and compute the performance metrics
  with torch.no_grad():
    for X, y in dataloader:
      X = X.to(device)
      y = y.to(device)
      pred1 = model1(X)
      pred2 = model2(X)
      pred3 = model3(X)
      pred = (pred1+pred2+pred3)/3
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size

  # print the performance metrics
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# define the loss function
loss_fn = nn.CrossEntropyLoss()

# evaluate the first model
ensemble_test_loop(dataloaders['testing'], model1, model2, model3, loss_fn)

Test Error: 
 Accuracy: 88.5%, Avg loss: 0.413374 

